# SentEval usage example

* Clone repo from FAIR github
```
    git clone https://github.com/facebookresearch/SentEval.git
    cd SentEval/
```
* Dependencies:
    * Python 2/3 with NumPy/SciPy
    * Pytorch
    * scikit-learn>=0.18.0

* Install senteval
```
    python setup.py install
```
* Download datasets (it takes some time...)
    * these are downstream tasks
    * new Senteval also has probing tasks (https://github.com/facebookresearch/SentEval/tree/master/data/probing) for evaluating linguistic properties of your embeddings. 
```
    cd data/downstream/
    ./get_transfer_data.bash
```
* Download pretained Glove embeddings:

```
    mkdir pretrained
    cd pretrained
    wget http://nlp.stanford.edu/data/glove.840B.300d.zip
   
```

* The following code evaluates Glove pretrained embeddings on different NLP downstream tasks.

In [3]:
# Copyright (c) 2017-present, Facebook, Inc.
# All rights reserved.
#
# This source code is licensed under the license found in the
# LICENSE file in the root directory of this source tree.
#

from __future__ import absolute_import, division, unicode_literals

import sys
import numpy as np
import logging
import sklearn
import SentEval.examples.data as data

from gensim.models import Word2Vec

# Set PATHs
# path to senteval
PATH_TO_SENTEVAL = './SentEval/'
# path to the NLP datasets 
PATH_TO_DATA = './SentEval/data/'


# import SentEval
sys.path.insert(0, PATH_TO_SENTEVAL)
import senteval


def prepare(params, samples):
    # load model:
    sg_model = Word2Vec.load('gensim_skipgram_model')
    params.word_vec = sg_model.wv
    # dimensionality of glove embeddings
    params.wvec_dim = 300
    return

def batcher(params, batch):
    """
    In this example we use the average of word embeddings as a sentence representation.
    Each batch consists of one vector for sentence.
    Here you can process each sentence of the batch, 
    or a complete batch (you may need masking for that).
    
    """
    # if a sentence is empty dot is set to be the only token
    # you can change it into NULL dependening in your model
    batch = [sent if sent != [] else ['.'] for sent in batch]
    embeddings = []

    for sent in batch:
        sentvec = []
        # the format of a sentence is a lists of words (tokenized and lowercased)
        for word in sent:
            if word in params.word_vec:
                # [number of words, embedding dimensionality]
                sentvec.append(params.word_vec[word])
        if not sentvec:
            vec = np.zeros(params.wvec_dim)
            # [number of words, embedding dimensionality]
            sentvec.append(vec)
        # average of word embeddings for sentence representation
        # [embedding dimansionality]
        sentvec = np.mean(sentvec, 0)
        embeddings.append(sentvec)
    # [batch size, embedding dimensionality]
    embeddings = np.vstack(embeddings)
    return embeddings


# Set params for SentEval
# we use logistic regression (usepytorch: Fasle) and kfold 10
# In this dictionary you can add extra information that you model needs for initialization
# for example the path to a dictionary of indices, of hyper parameters
# this dictionary is passed to the batched and the prepare fucntions
params_senteval = {'task_path': PATH_TO_DATA, 'usepytorch': False, 'kfold': 10}
# this is the config for the NN classifier but we are going to use scikit-learn logistic regression with 10 kfold
# usepytorch = False 
#params_senteval['classifier'] = {'nhid': 0, 'optim': 'rmsprop', 'batch_size': 128,
#                                 'tenacity': 3, 'epoch_size': 2}

# Set up logger
logging.basicConfig(format='%(asctime)s : %(message)s', level=logging.DEBUG)

if __name__ == "__main__":
    se = senteval.engine.SE(params_senteval, batcher, prepare)
    
    # here you define the NLP taks that your embedding model is going to be evaluated
    # in (https://arxiv.org/abs/1802.05883) we use the following :
    # SICKRelatedness (Sick-R) needs torch cuda to work (even when using logistic regression), 
    # but STS14 (semantic textual similarity) is a similar type of semantic task
    transfer_tasks = ['MR', 'CR', 'MPQA', 'SUBJ', 'SST2', 'TREC',
                      'MRPC', 'SICKEntailment', 'STS14']
    # senteval prints the results and returns a dictionary with the scores
    results = se.eval(transfer_tasks)
    print(results)

2018-05-23 10:58:22,839 : ***** Transfer task : MR *****


2018-05-23 10:58:22,843 : loading Word2Vec object from gensim_skipgram_model
2018-05-23 10:58:22,845 : {'kw': {}, 'uri': 'gensim_skipgram_model', 'mode': 'rb'}
2018-05-23 10:58:22,848 : encoding_wrapper: {'fileobj': <_io.BufferedReader name='gensim_skipgram_model'>, 'errors': 'strict', 'encoding': None, 'mode': 'rb'}
2018-05-23 10:58:23,193 : loading wv recursively from gensim_skipgram_model.wv.* with mmap=None
2018-05-23 10:58:23,194 : loading vectors from gensim_skipgram_model.wv.vectors.npy with mmap=None
2018-05-23 10:58:23,246 : setting ignored attribute vectors_norm to None
2018-05-23 10:58:23,248 : loading vocabulary recursively from gensim_skipgram_model.vocabulary.* with mmap=None
2018-05-23 10:58:23,250 : loading trainables recursively from gensim_skipgram_model.trainables.* with mmap=None
2018-05-23 10:58:23,251 : loading syn1neg from gensim_skipgram_model.trainables.syn1neg.npy with mmap=None
2018-05-23 10:58:23,305

{'MPQA': {'acc': 87.27, 'ndev': 10606, 'ntest': 10606, 'devacc': 87.46}, 'STS14': {'deft-forum': {'nsamples': 450, 'pearson': (0.14699366601892444, 0.00176880369247389), 'spearman': SpearmanrResult(correlation=0.26440064569769295, pvalue=1.2348695718660246e-08)}, 'tweet-news': {'nsamples': 750, 'pearson': (0.30105459770846466, 3.5389344064642394e-17), 'spearman': SpearmanrResult(correlation=0.3923990152618318, pvalue=5.115931167356124e-29)}, 'headlines': {'nsamples': 750, 'pearson': (0.17920334152225456, 7.830740336534862e-07), 'spearman': SpearmanrResult(correlation=0.2645610258875473, pvalue=1.7736331937823493e-13)}, 'deft-news': {'nsamples': 300, 'pearson': (0.6004587733348685, 8.883026903479862e-31), 'spearman': SpearmanrResult(correlation=0.5759303339588969, pvalue=6.643351477103063e-28)}, 'all': {'pearson': {'wmean': 0.34905251512983626, 'mean': 0.360722551009612}, 'spearman': {'wmean': 0.4184975123179476, 'mean': 0.42396767004069225}}, 'OnWN': {'nsamples': 750, 'pearson': (0.516